In [1]:
!pip install ray[rllib]==1.0.1
!pip install pygame

In [2]:
import os.path

if not os.path.exists("/home/jovyan/work/PyGame-Learning-Environment"):
    !git clone https://github.com/ntasfi/PyGame-Learning-Environment.git /home/jovyan/work/PyGame-Learning-Environment
        
try:
    import ple
except ImportError:
    !pip install -e /home/jovyan/work/PyGame-Learning-Environment

if not os.path.exists("/home/jovyan/work/gym-ple"):
    !git clone https://github.com/lusob/gym-ple.git /home/jovyan/work/gym-ple

try:
    import gym_ple
except ImportError:
    !pip install -e /home/jovyan/work/gym-ple

pygame 2.0.1 (SDL 2.0.14, Python 3.7.9)
Hello from the pygame community. https://www.pygame.org/contribute.html
couldn't import doomish
Couldn't import doom


In [ ]:
import numpy as np
import glob
import ray
import ray.rllib.evaluation.rollout_worker as rw
import ray.rllib.agents.dqn as dqn
import ray.rllib.agents.dqn.apex as apex
from ray.tune.logger import pretty_print

from gym_ple.ple_env import PLEEnv
from ray.tune.registry import register_env

checkpoint_root = "tmp/dqn/catcher"

# Get Ray started
ray.shutdown()
ray.init(object_store_memory=4*1024**3,include_dashboard=False)

#['Catcher', 'MonsterKong', 'FlappyBird', 'PixelCopter', 'PuckWorld', 'RaycastMaze', 'Snake', 'WaterWorld']:
screen_size = 42
def env_creator(env_config):
    rewards = {
                "positive": 1.0,
                "negative": -1.0,
                "tick": 0.05,
                "loss": -5.0,
                "win": 5.0
            }
    return PLEEnv(game_name='Catcher',display_screen=False, width=screen_size,
                  height=screen_size)  # return an env instance

register_env("PixelCopter-v0", env_creator)

# Setup configuration parameters
niter = 10
rlconfig = {"timesteps_per_iteration": 8000,
            "model": {"dim": screen_size,
                      "conv_filters": [[16, [4, 4], 2], [32, [4, 4], 2], [512, [11, 11], 1]],
                      "framestack": True,
                      #"use_lstm": True,
                      #"use_attention": False,
                      #"lstm_cell_size": 64,
                      #"max_seq_len": 999999,
                     },
            "evaluation_num_workers": 4,
            "evaluation_interval": 1,
            "evaluation_num_episodes": 1000,
            "gamma": 0.97,
            "lr": 4e-5,
            "evaluation_config": {
                "explore": False
            },
            "exploration_config": {
                "type": "EpsilonGreedy",
                "initial_epsilon": 0.6,
                "final_epsilon": 0.01,
                "epsilon_timesteps": 2000000,  # time from initial to final
            },
           }

cpuconfig = {"num_envs_per_worker": 16,
             "num_gpus": 1,
             "num_workers": 4,
            }

# Update default config
config = dqn.DEFAULT_CONFIG.copy()
config.update(rlconfig)
config.update(cpuconfig)

# Construct trainer
agent = dqn.DQNTrainer(config=config, env="PixelCopter-v0")

prior_saves = glob.glob(checkpoint_root + '/checkpoint_*/checkpoint-*[0-9]')
if len(prior_saves) != 0:
    highest = max(prior_saves, key=lambda x: int(x.split('-',1)[1].split('.',1)[0]))
    agent.restore(highest)

# Get policy network and print summary
policy = agent.get_policy()
model = policy.model
print(model.base_model.summary())

# Run training iterations
for i in range(niter):
    # Perform one iteration of training the policy
    result = agent.train()
    file_name = agent.save(checkpoint_root)
    print("Iteration %02d -- Mean Reward: %2.4f -- Eval Reward %2.4f" % (i,result["episode_reward_mean"],result["evaluation"]["episode_reward_mean"]) )

Instructions for updating:
non-resource variables are not supported in the long term


2021-03-07 03:18:22,123	INFO trainer.py:592 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
2021-03-07 03:18:22,125	INFO trainer.py:619 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=48794) WARNING:tensorflow:From /opt/conda/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:88: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=48794) Instructions for updating:
(pid=48794) non-resource variables are not supported in the long term
(pid=48833) WARNING:tensorflow:From /opt/conda/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:88: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=48833) Instructions for updating:
(pid=48833) non-resource variables are not supported in the long term


(pid=48834) pygame 2.0.1 (SDL 2.0.14, Python 3.7.9)
(pid=48834) Hello from the pygame community. https://www.pygame.org/contribute.html
(pid=48834) couldn't import doomish
(pid=48834) Couldn't import doom
(pid=48794) pygame 2.0.1 (SDL 2.0.14, Python 3.7.9)
(pid=48794) Hello from the pygame community. https://www.pygame.org/contribute.html
(pid=48794) couldn't import doomish
(pid=48794) Couldn't import doom
(pid=48833) pygame 2.0.1 (SDL 2.0.14, Python 3.7.9)
(pid=48833) Hello from the pygame community. https://www.pygame.org/contribute.html
(pid=48833) couldn't import doomish
(pid=48833) Couldn't import doom
(pid=48813) pygame 2.0.1 (SDL 2.0.14, Python 3.7.9)
(pid=48813) Hello from the pygame community. https://www.pygame.org/contribute.html
(pid=48813) couldn't import doomish
(pid=48813) Couldn't import doom


(pid=48834) WARNING:tensorflow:From /opt/conda/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1635: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
(pid=48834) Instructions for updating:
(pid=48834) If using Keras pass *_constraint arguments to layers.
(pid=48794) WARNING:tensorflow:From /opt/conda/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1635: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
(pid=48794) Instructions for updating:
(pid=48794) If using Keras pass *_constraint arguments to layers.
(pid=48833) WARNING:tensorflow:From /opt/conda/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1635: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resou

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


(pid=48837) WARNING:tensorflow:From /opt/conda/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:88: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=48837) Instructions for updating:
(pid=48837) non-resource variables are not supported in the long term
(pid=48842) WARNING:tensorflow:From /opt/conda/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:88: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=48842) Instructions for updating:
(pid=48842) non-resource variables are not supported in the long term
(pid=48816) WARNING:tensorflow:From /opt/conda/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:88: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=48816) Instructions for updating:

(pid=48842) pygame 2.0.1 (SDL 2.0.14, Python 3.7.9)
(pid=48842) Hello from the pygame community. https://www.pygame.org/contribute.html
(pid=48842) couldn't import doomish
(pid=48842) Couldn't import doom
(pid=48837) pygame 2.0.1 (SDL 2.0.14, Python 3.7.9)
(pid=48837) Hello from the pygame community. https://www.pygame.org/contribute.html
(pid=48837) couldn't import doomish
(pid=48837) Couldn't import doom
(pid=48822) pygame 2.0.1 (SDL 2.0.14, Python 3.7.9)
(pid=48822) Hello from the pygame community. https://www.pygame.org/contribute.html
(pid=48822) couldn't import doomish
(pid=48822) Couldn't import doom
(pid=48816) pygame 2.0.1 (SDL 2.0.14, Python 3.7.9)
(pid=48816) Hello from the pygame community. https://www.pygame.org/contribute.html
(pid=48816) couldn't import doomish
(pid=48816) Couldn't import doom


(pid=48842) WARNING:tensorflow:From /opt/conda/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1635: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
(pid=48842) Instructions for updating:
(pid=48842) If using Keras pass *_constraint arguments to layers.
(pid=48837) WARNING:tensorflow:From /opt/conda/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1635: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
(pid=48837) Instructions for updating:
(pid=48837) If using Keras pass *_constraint arguments to layers.
(pid=48822) WARNING:tensorflow:From /opt/conda/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1635: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resou

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
observations (InputLayer)    [(None, 42, 42, 3)]       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 21, 21, 16)        784       
_________________________________________________________________
conv2 (Conv2D)               (None, 11, 11, 32)        8224      
_________________________________________________________________
conv_out (Conv2D)            (None, 1, 1, 256)         991488    
_________________________________________________________________
lambda (Lambda)              (None, 256)               0         
_________________________________________________________________
value_out (Dense)            (None, 1)                 257       
Total params: 1,000,753
Trainable params: 1,000,753
Non-trainable params: 0
___________________________________________________

(pid=48834) WARNING:tensorflow:From /opt/conda/lib/python3.7/site-packages/ray/rllib/policy/tf_policy.py:836: Variable.load (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
(pid=48834) Instructions for updating:
(pid=48834) Prefer Variable.assign which has equivalent behavior in 2.X.
(pid=48794) WARNING:tensorflow:From /opt/conda/lib/python3.7/site-packages/ray/rllib/policy/tf_policy.py:836: Variable.load (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
(pid=48794) Instructions for updating:
(pid=48794) Prefer Variable.assign which has equivalent behavior in 2.X.
(pid=48833) WARNING:tensorflow:From /opt/conda/lib/python3.7/site-packages/ray/rllib/policy/tf_policy.py:836: Variable.load (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
(pid=48833) Instructions for updating:
(pid=48833) Prefer Variable.assign which has equivalent behavior in 2.X.
(pid=48813

In [ ]:
import imageio
import base64
from IPython import display

def gif_animate(frames,fps=5):
    seq = []
    for i in range(len(frames)):
        fig, ax = plt.subplots(figsize=(5,5))
        ax.imshow(frames[i])
        plt.axis('off')
        fig.canvas.draw()
        image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
        plt.close(fig)
        seq.append(image.reshape(fig.canvas.get_width_height()[::-1] + (3,)))

    imageio.mimsave('./anim.gif', seq, fps=fps)
    return base64.b64encode(open('./anim.gif','rb').read())


In [ ]:
import matplotlib.pyplot as plt
from ray.rllib.models.preprocessors import get_preprocessor

# Setup separate environment
env = env_creator({})

# Get function that maps combinatorial env state to one-hot state
prep = get_preprocessor(env.observation_space)(env.observation_space)    
    
# Use trained policy (without exploration) to play N episodes
N = 10
total_reward = 0
gifs = []
for i in range(N):
    # run until episode ends
    episode_reward = 0
    done = False
    obs = env.reset()
    if (N<=10):
        episode = []
        episode.append(obs)
    while not done:
        action = agent.compute_action(prep.transform(obs),explore=False)
        #action = agent.compute_action(obs,explore=False)
        obs, reward, done, info = env.step(action)
        if (N<=10):
            episode.append(obs)
        episode_reward += reward
        
    if (N<=10):
        gifs.append(gif_animate(episode,fps=15))
        
    total_reward += episode_reward

# Make HTML movie
s = '<table><tr>'
for i in range(len(gifs)):
    s = s + '<td><img width="100" src="data:image/gif;base64,{0}"/></td>'.format(gifs[i].decode('ascii'))
s = s + '</tr></table>'
display.display(display.HTML(s))              

In [ ]:


"""
Classic cart-pole system implemented by Rich Sutton et al.
Copied from http://incompleteideas.net/sutton/book/code/pole.c
permalink: https://perma.cc/C9ZM-652R
"""

import math
import gym
from gym import spaces, logger
from gym.utils import seeding
import numpy as np


class CartPoleEnv(gym.Env):
    """
    Description:
        A pole is attached by an un-actuated joint to a cart, which moves along
        a frictionless track. The pendulum starts upright, and the goal is to
        prevent it from falling over by increasing and reducing the cart's
        velocity.
    Source:
        This environment corresponds to the version of the cart-pole problem
        described by Barto, Sutton, and Anderson
    Observation:
        Type: Box(4)
        Num     Observation               Min                     Max
        0       Cart Position             -4.8                    4.8
        1       Cart Velocity             -Inf                    Inf
        2       Pole Angle                -0.418 rad (-24 deg)    0.418 rad (24 deg)
        3       Pole Angular Velocity     -Inf                    Inf
    Actions:
        Type: Discrete(2)
        Num   Action
        0     Push cart to the left
        1     Push cart to the right
        Note: The amount the velocity that is reduced or increased is not
        fixed; it depends on the angle the pole is pointing. This is because
        the center of gravity of the pole increases the amount of energy needed
        to move the cart underneath it
    Reward:
        Reward is 1 for every step taken, including the termination step
    Starting State:
        All observations are assigned a uniform random value in [-0.05..0.05]
    Episode Termination:
        Pole Angle is more than 12 degrees.
        Cart Position is more than 2.4 (center of the cart reaches the edge of
        the display).
        Episode length is greater than 200.
        Solved Requirements:
        Considered solved when the average return is greater than or equal to
        195.0 over 100 consecutive trials.
    """

    metadata = {
        'render.modes': ['human', 'rgb_array'],
        'video.frames_per_second': 50
    }

    def __init__(self):
        self.gravity = 9.8
        self.masscart = 1.0
        self.masspole = 0.1
        self.total_mass = (self.masspole + self.masscart)
        self.length = 0.5  # actually half the pole's length
        self.polemass_length = (self.masspole * self.length)
        self.force_mag = 10.0
        self.tau = 0.02  # seconds between state updates
        self.kinematics_integrator = 'euler'

        # Angle at which to fail the episode
        self.theta_threshold_radians = 12 * 2 * math.pi / 360
        self.x_threshold = 2.4

        # Angle limit set to 2 * theta_threshold_radians so failing observation
        # is still within bounds.
        high = np.array([self.x_threshold * 2,
                         np.finfo(np.float32).max,
                         self.theta_threshold_radians * 2,
                         np.finfo(np.float32).max],
                        dtype=np.float32)

        self.action_space = spaces.Discrete(2)
        self.observation_space = spaces.Box(-high, high, dtype=np.float32)

        self.seed()
        self.viewer = None
        self.state = None

        self.steps_beyond_done = None

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def step(self, action):
        err_msg = "%r (%s) invalid" % (action, type(action))
        assert self.action_space.contains(action), err_msg

        x, x_dot, theta, theta_dot = self.state
        force = self.force_mag if action == 1 else -self.force_mag
        costheta = math.cos(theta)
        sintheta = math.sin(theta)

        # For the interested reader:
        # https://coneural.org/florian/papers/05_cart_pole.pdf
        temp = (force + self.polemass_length * theta_dot ** 2 * sintheta) / self.total_mass
        thetaacc = (self.gravity * sintheta - costheta * temp) / (self.length * (4.0 / 3.0 - self.masspole * costheta ** 2 / self.total_mass))
        xacc = temp - self.polemass_length * thetaacc * costheta / self.total_mass

        if self.kinematics_integrator == 'euler':
            x = x + self.tau * x_dot
            x_dot = x_dot + self.tau * xacc
            theta = theta + self.tau * theta_dot
            theta_dot = theta_dot + self.tau * thetaacc
        else:  # semi-implicit euler
            x_dot = x_dot + self.tau * xacc
            x = x + self.tau * x_dot
            theta_dot = theta_dot + self.tau * thetaacc
            theta = theta + self.tau * theta_dot

        self.state = (x, x_dot, theta, theta_dot)

        done = bool(
            x < -self.x_threshold
            or x > self.x_threshold
            or theta < -self.theta_threshold_radians
            or theta > self.theta_threshold_radians
        )

        if not done:
            reward = 1.0
        elif self.steps_beyond_done is None:
            # Pole just fell!
            self.steps_beyond_done = 0
            reward = 1.0
        else:
            if self.steps_beyond_done == 0:
                logger.warn(
                    "You are calling 'step()' even though this "
                    "environment has already returned done = True. You "
                    "should always call 'reset()' once you receive 'done = "
                    "True' -- any further steps are undefined behavior."
                )
            self.steps_beyond_done += 1
            reward = 0.0

        return np.array(self.state), reward, done, {}

    def reset(self):
        self.state = self.np_random.uniform(low=-0.05, high=0.05, size=(4,))
        self.steps_beyond_done = None
        return np.array(self.state)

    def render(self, mode='human'):
        screen_width = 600
        screen_height = 400

        world_width = self.x_threshold * 2
        scale = screen_width/world_width
        carty = 100  # TOP OF CART
        polewidth = 10.0
        polelen = scale * (2 * self.length)
        cartwidth = 50.0
        cartheight = 30.0

        if self.viewer is None:
            from gym.envs.classic_control import rendering
            self.viewer = rendering.Viewer(screen_width, screen_height)
            l, r, t, b = -cartwidth / 2, cartwidth / 2, cartheight / 2, -cartheight / 2
            axleoffset = cartheight / 4.0
            cart = rendering.FilledPolygon([(l, b), (l, t), (r, t), (r, b)])
            self.carttrans = rendering.Transform()
            cart.add_attr(self.carttrans)
            self.viewer.add_geom(cart)
            l, r, t, b = -polewidth / 2, polewidth / 2, polelen - polewidth / 2, -polewidth / 2
            pole = rendering.FilledPolygon([(l, b), (l, t), (r, t), (r, b)])
            pole.set_color(.8, .6, .4)
            self.poletrans = rendering.Transform(translation=(0, axleoffset))
            pole.add_attr(self.poletrans)
            pole.add_attr(self.carttrans)
            self.viewer.add_geom(pole)
            self.axle = rendering.make_circle(polewidth/2)
            self.axle.add_attr(self.poletrans)
            self.axle.add_attr(self.carttrans)
            self.axle.set_color(.5, .5, .8)
            self.viewer.add_geom(self.axle)
            self.track = rendering.Line((0, carty), (screen_width, carty))
            self.track.set_color(0, 0, 0)
            self.viewer.add_geom(self.track)

            self._pole_geom = pole

        if self.state is None:
            return None

        # Edit the pole polygon vertex
        pole = self._pole_geom
        l, r, t, b = -polewidth / 2, polewidth / 2, polelen - polewidth / 2, -polewidth / 2
        pole.v = [(l, b), (l, t), (r, t), (r, b)]

        x = self.state
        cartx = x[0] * scale + screen_width / 2.0  # MIDDLE OF CART
        self.carttrans.set_translation(cartx, carty)
        self.poletrans.set_rotation(-x[2])

        return self.viewer.render(return_rgb_array=mode == 'rgb_array')

    def close(self):
        if self.viewer:
            self.viewer.close()
            self.viewer = None

In [ ]:
import ray
import ray.rllib.agents.ppo as ppo
import json

checkpoint_root = "tmp/ppo/cart"

info = ray.init(ignore_reinit_error=True)
SELECT_ENV = "CartPole-v1"                      # Specifies the OpenAI Gym environment for Cart Pole
N_ITER = 10                                     # Number of training runs.

config = ppo.DEFAULT_CONFIG.copy()              # PPO's default configuration. See the next code cell.
config["log_level"] = "WARN"                    # Suppress too many messages, but try "INFO" to see what can be printed.

# Other settings we might adjust:
config["num_workers"] = 1                       # Use > 1 for using more CPU cores, including over a cluster
config["num_sgd_iter"] = 10                     # Number of SGD (stochastic gradient descent) iterations per training minibatch.
                                                # I.e., for each minibatch of data, do this many passes over it to train. 
config["sgd_minibatch_size"] = 250              # The amount of data records per minibatch
config["model"]["fcnet_hiddens"] = [100, 50]    #
config["num_cpus_per_worker"] = 0  # This avoids running out of resources in the notebook environment when this cell is re-executed

agent = ppo.PPOTrainer(config, env=SELECT_ENV)

results = []
episode_data = []
episode_json = []

for n in range(N_ITER):
    result = agent.train()
    results.append(result)
    
    episode = {'n': n, 
               'episode_reward_min': result['episode_reward_min'], 
               'episode_reward_mean': result['episode_reward_mean'], 
               'episode_reward_max': result['episode_reward_max'],  
               'episode_len_mean': result['episode_len_mean']}
    
    episode_data.append(episode)
    episode_json.append(json.dumps(episode))
    file_name = agent.save(checkpoint_root)
    
    print(f'{n:3d}: Min/Mean/Max reward: {result["episode_reward_min"]:8.4f}/{result["episode_reward_mean"]:8.4f}/{result["episode_reward_max"]:8.4f}. Checkpoint saved to {file_name}')

In [ ]:
checkpoint_root = "tmp/ppo/taxi"

info = ray.init(ignore_reinit_error=True)
SELECT_ENV = "Taxi-v3"                      # Specifies the OpenAI Gym environment for Taxi-v3
N_ITER = 10                                     # Number of training runs.

config = ppo.DEFAULT_CONFIG.copy()              # PPO's default configuration. See the next code cell.
config["log_level"] = "WARN"                    # Suppress too many messages, but try "INFO" to see what can be printed.

# Other settings we might adjust:
config["num_workers"] = 1                       # Use > 1 for using more CPU cores, including over a cluster
config["num_sgd_iter"] = 10                     # Number of SGD (stochastic gradient descent) iterations per training minibatch.
                                                # I.e., for each minibatch of data, do this many passes over it to train. 
config["sgd_minibatch_size"] = 250              # The amount of data records per minibatch
config["model"]["fcnet_hiddens"] = [100, 50]    #
config["num_cpus_per_worker"] = 0  # This avoids running out of resources in the notebook environment when this cell is re-executed

agent = ppo.PPOTrainer(config, env=SELECT_ENV)

results = []
episode_data = []
episode_json = []

for n in range(N_ITER):
    result = agent.train()
    results.append(result)
    
    episode = {'n': n, 
               'episode_reward_min': result['episode_reward_min'], 
               'episode_reward_mean': result['episode_reward_mean'], 
               'episode_reward_max': result['episode_reward_max'],  
               'episode_len_mean': result['episode_len_mean']}
    
    episode_data.append(episode)
    episode_json.append(json.dumps(episode))
    file_name = agent.save(checkpoint_root)
    
    print(f'{n:3d}: Min/Mean/Max reward: {result["episode_reward_min"]:8.4f}/{result["episode_reward_mean"]:8.4f}/{result["episode_reward_max"]:8.4f}. Checkpoint saved to {file_name}')